<a href="https://colab.research.google.com/github/scarlettyu2023/AI_agent_workshop/blob/main/Topic%203/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Goals
Know how and why to run an Ollama LLM server

Know how to run large commerical models in LangGraph

Understand how LLMs call tools using the OpenAI format

Understand how to most efficiently define and use LangGraph tools

#1. Set up an Ollama server running Llama 3.2-1B  on either your laptop (if it has a GPU) or on CoLab (if not).  

Create two copies of the  llama_mmlu_eval.py code from Topic 1 and modify them so that each runs on a single different topic.  Use the command line time shell function  to measure how long each takes to run in real clock time.  Then modify the programs so they use Ollama.  Start the Ollama server running and then time both sequential and parallel execution of the two programs.  For sequential execution, run

time { python program1.py ; python program2.py }
Next, time them running in parallel:

time { python program1.py & python program2.py & wait; }
Write in the README.md file in your portfolio what you observed.



In [1]:
!sudo apt-get update -qq
!sudo apt-get install -y zstd

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 116 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (509 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf

In [2]:
%%bash
set -e
sudo pkill -f "ollama" || true
sudo fuser -k 11434/tcp || true
sleep 1
ps aux | grep -i ollama | head -n 20


root        2511  0.0  0.0   6484  2412 ?        S    18:00   0:00 grep -i ollama


In [4]:
!pip -q install datasets requests tqdm
!sudo apt-get -qq update
!sudo apt-get -qq install -y pciutils lsb-release curl
!curl -fsSL https://ollama.com/install.sh | sh


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package pci.ids.
(Reading database ... 121710 files and directories currently installed.)
Preparing to unpack .../pci.ids_0.0~2022.01.22-1ubuntu0.1_all.deb ...
Unpacking pci.ids (0.0~2022.01.22-1ubuntu0.1) ...
Selecting previously unselected package libpci3:amd64.
Preparing to unpack .../libpci3_1%3a3.7.0-6_amd64.deb ...
Unpacki

In [5]:
%%bash
set -e
nohup ollama serve > ollama.log 2>&1 &
sleep 2

# Check it is running
ps aux | grep -i "ollama serve" | grep -v grep || true

# Check port is open
curl -s http://127.0.0.1:11434/api/tags | head -n 5 || true

echo "---- last 30 lines of ollama.log ----"
tail -n 30 ollama.log

root        3720  2.0  0.2 1858408 32264 ?       Sl   18:02   0:00 ollama serve
{"models":[]}---- last 30 lines of ollama.log ----
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIBVm6q577PSYiyzy60GAluOUX3l52Un7e6A11sQ3+Qr0

time=2026-02-03T18:02:02.406Z level=INFO source=routes.go:1631 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GGML_VK_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://loca

In [6]:
# Start ollama in the background
!nohup ollama serve > ollama.log 2>&1 &

# Quick sanity check: server should answer (may take a few seconds right after start)
!curl -s http://localhost:11434/api/tags | head

# Pull the model (about ~1.3GB)
!ollama pull llama3.2:1b


{"models":[]}


In [8]:
pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.3.4
    Uninstalling huggingface_hub-1.3.4:
      Successfully uninstalled huggingface_hub-1.3.4


In [9]:
!hf version

1.3.7


In [10]:
!git config --global credential.helper store
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? [y/N]: y
Token is valid (permission: read).
The token `scarlettyucs6501` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.ca

In [11]:
!ls

ollama.log  sample_data


In [14]:
!time { python llama_mmlu_eval_topic1.py ; python llama_mmlu_eval_topic2.py ; }


Llama 3.2-1B MMLU Evaluation (Quantized)

Environment Check
âœ“ Running in Google Colab
âœ“ Platform: Linux (x86_64)
âœ“ GPU Available: Tesla T4
âœ“ GPU Memory: 15.83 GB
âœ“ Quantization disabled - loading full precision model
âš ï¸  Could not check Hugging Face authentication

Configuration
Model: allenai/OLMo-2-0425-1B
Device: cuda
Quantization: None (full precision)
Expected memory: ~2.5 GB (FP16)
Number of subjects: 1


Loading model allenai/OLMo-2-0425-1B...
Device: cuda
âœ“ Tokenizer loaded
Loading model (this may take 2-3 minutes)...
Loading weights: 100% 179/179 [00:02<00:00, 64.16it/s, Materializing param=model.norm.weight] 
âœ“ Model loaded successfully!
  Model device: cuda:0
  Model dtype: torch.float16
  GPU Memory: 2.97 GB allocated, 2.97 GB reserved

Starting evaluation on 1 subjects


Progress: 1/1 subjects

Evaluating subject: abstract_algebra
Testing abstract_algebra: 100% 100/100 [00:03<00:00, 32.27it/s]
âœ“ Result: 32/100 correct = 32.00%

EVALUATION SUMMARY
Model

In [13]:
!time { python llama_mmlu_eval_topic1.py & python llama_mmlu_eval_topic2.py & wait ; }


Llama 3.2-1B MMLU Evaluation (Quantized)

Environment Check

Llama 3.2-1B MMLU Evaluation (Quantized)

Environment Check
âœ“ Running in Google Colab
âœ“ Running in Google Colab
âœ“ Platform: Linux (x86_64)
âœ“ Platform: Linux (x86_64)
âœ“ GPU Available: Tesla T4
âœ“ GPU Memory: 15.83 GB
âœ“ Quantization disabled - loading full precision model
âš ï¸  Could not check Hugging Face authentication

Configuration
Model: allenai/OLMo-2-0425-1B
Device: cuda
Quantization: None (full precision)
Expected memory: ~2.5 GB (FP16)
Number of subjects: 1


Loading model allenai/OLMo-2-0425-1B...
Device: cuda
âœ“ GPU Available: Tesla T4
âœ“ GPU Memory: 15.83 GB
âœ“ Quantization disabled - loading full precision model
âš ï¸  Could not check Hugging Face authentication

Configuration
Model: allenai/OLMo-2-0425-1B
Device: cuda
Quantization: None (full precision)
Expected memory: ~2.5 GB (FP16)
Number of subjects: 1


Loading model allenai/OLMo-2-0425-1B...
Device: cuda
config.json: 100% 623/623 [00:00<0

#2. We will use GPT-4o Mini for the rest of the topic because it is very good at tool use and inexpensive to run (if not completely free) - it should cost less than $1 to complete this topic.  See the full list of OpenAI pricing here.



Pass the secret key

In [15]:
from google.colab import userdata
import os
# Make it available like system env vars
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [16]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "Say: Working!"}],
 max_tokens=5)
print(response.choices[0].message.content)

Working! How can I


# Download the code sample for manual tool handling and make sure it runs on your setup.  

In [17]:
!python manual_tool_handling.py

TEST 1: Query requiring tool
User: What's the weather like in San Francisco?

--- Iteration 1 ---
LLM wants to call 1 tool(s)
  Tool: get_weather
  Args: {'location': 'San Francisco'}
  Result: Sunny, 72Â°F

--- Iteration 2 ---
Assistant: The weather in San Francisco is sunny, with a temperature of 72°F.


TEST 2: Query not requiring tool
User: Say hello!

--- Iteration 1 ---
Assistant: Hello! How can I assist you today?


TEST 3: Multiple tool calls
User: What's the weather in New York and London?

--- Iteration 1 ---
LLM wants to call 2 tool(s)
  Tool: get_weather
  Args: {'location': 'New York'}
  Result: Cloudy, 55Â°F
  Tool: get_weather
  Args: {'location': 'London'}
  Result: Rainy, 48Â°F

--- Iteration 2 ---
Assistant: The current weather is as follows:
- **New York:** Cloudy, 55°F
- **London:** Rainy, 48°F



# Modify the code to define a calculator tool that includes geometric functions. Although there are many calculator tools already built into LangGraph, you should define your own from scratch. To parse the input to your tool use json.loads and to format the output from the tool use json.dumps (if needed). For the calculator itself, you could use ast.literal_eval or numexpr. Add examples of the system output to your portfolio. Note: if you find that gpt-4.1-mini insists on trying to do calculations itself instead of using your calculator tool, then explore these strategies for forcing an LLM to use tools and write about it in your README.md.

In [18]:
"""
Manual Tool Calling Exercise
Students will see how tool calling works under the hood.
"""

import json
from openai import OpenAI

# ============================================
# PART 1: Define Your Tools
# ============================================

def get_weather(location: str) -> str:
    """Get the current weather for a location"""
    # Simulated weather data
    weather_data = {
        "San Francisco": "Sunny, 72Â°F",
        "New York": "Cloudy, 55Â°F",
        "London": "Rainy, 48Â°F",
        "Tokyo": "Clear, 65Â°F"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

import ast
import math

def calculator(operation: str, operands: list) -> str:
    """
    Simple calculator with geometric functions.
    Returns JSON string.
    """
    try:
        if operation == "add":
            result = sum(operands)

        elif operation == "subtract":
            result = operands[0] - operands[1]

        elif operation == "multiply":
            result = operands[0] * operands[1]

        elif operation == "divide":
            result = operands[0] / operands[1]

        elif operation == "area_circle":
            r = operands[0]
            result = math.pi * r * r

        elif operation == "circumference_circle":
            r = operands[0]
            result = 2 * math.pi * r

        elif operation == "area_rectangle":
            w, h = operands
            result = w * h

        elif operation == "hypotenuse":
            a, b = operands
            result = math.sqrt(a * a + b * b)

        else:
            return json.dumps({"error": "Unknown operation"})

        return json.dumps({"result": result})

    except Exception as e:
        return json.dumps({"error": str(e)})

# ============================================
# PART 2: Describe Tools to the LLM
# ============================================

# This is the JSON schema that tells the LLM what tools exist
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco"
                    }
                },
                "required": ["location"]
            }
        }
    },
    # TODO: Students will add a second tool here (e.g., calculator)
    {
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Calculator with basic arithmetic and geometric functions",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "description": "Operation name: add, subtract, multiply, divide, area_circle, circumference_circle, area_rectangle, hypotenuse"
                },
                "operands": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "List of numbers for the operation"
                }
            },
            "required": ["operation", "operands"]
            }
        }
      }
    ]


# ============================================
# PART 3: The Agent Loop
# ============================================

def run_agent(user_query: str):
    """
    Simple agent that can use tools.
    Shows the manual loop that LangGraph automates.
    """

    # Initialize OpenAI client
    client = OpenAI()

    # Start conversation with user query
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the provided tools when needed."},
        {"role": "user", "content": user_query}
    ]

    print(f"User: {user_query}\n")

    # Agent loop - can iterate up to 5 times
    for iteration in range(5):
        print(f"--- Iteration {iteration + 1} ---")

        # Call the LLM
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,  # â† This tells the LLM what tools are available
            tool_choice="auto"  # Let the model decide whether to use tools
        )

        assistant_message = response.choices[0].message

        # Check if the LLM wants to call a tool
        if assistant_message.tool_calls:
            print(f"LLM wants to call {len(assistant_message.tool_calls)} tool(s)")

            # Add the assistant's response to messages
            messages.append(assistant_message)

            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                print(f"  Tool: {function_name}")
                print(f"  Args: {function_args}")

                # THIS IS THE MANUAL DISPATCH
                # In a real system, you'd use a dictionary lookup
                if function_name == "get_weather":
                    result = get_weather(**function_args)
                elif function_name == "calculator":
                    result = calculator(**function_args)
                else:
                    result = f"Error: Unknown function {function_name}"

                print(f"  Result: {result}")

                # Add the tool result back to the conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result
                })

            print()
            # Loop continues - LLM will see the tool results

        else:
            # No tool calls - LLM provided a final answer
            print(f"Assistant: {assistant_message.content}\n")
            return assistant_message.content

    return "Max iterations reached"


# ============================================
# PART 4: Test It
# ============================================

if __name__ == "__main__":
    # Test query that requires tool use
    print("="*60)
    print("TEST 1: Query requiring tool")
    print("="*60)
    run_agent("What's the weather like in San Francisco?")

    print("\n" + "="*60)
    print("TEST 2: Query not requiring tool")
    print("="*60)
    run_agent("Say hello!")

    print("\n" + "="*60)
    print("TEST 3: Multiple tool calls")
    print("="*60)
    run_agent("What's the weather in New York and London?")

TEST 1: Query requiring tool
User: What's the weather like in San Francisco?

--- Iteration 1 ---
LLM wants to call 1 tool(s)
  Tool: get_weather
  Args: {'location': 'San Francisco'}
  Result: Sunny, 72Â°F

--- Iteration 2 ---
Assistant: The weather in San Francisco is sunny with a temperature of 72°F.


TEST 2: Query not requiring tool
User: Say hello!

--- Iteration 1 ---
Assistant: Hello! How can I assist you today?


TEST 3: Multiple tool calls
User: What's the weather in New York and London?

--- Iteration 1 ---
LLM wants to call 2 tool(s)
  Tool: get_weather
  Args: {'location': 'New York'}
  Result: Cloudy, 55Â°F
  Tool: get_weather
  Args: {'location': 'London'}
  Result: Rainy, 48Â°F

--- Iteration 2 ---
Assistant: The current weather is as follows:

- **New York**: Cloudy, 55°F
- **London**: Rainy, 48°F



# Download the code sample for LangGraph tool handling and make sure it runs on your set.

In [19]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 5.2 MB/s eta 0:00:00


In [20]:
!python Langgraph_tool_handling.py

TEST 1: Query requiring tool
User: What's the weather like in San Francisco?

--- Iteration 1 ---
LLM wants to call 1 tool(s)
  Tool: get_weather
  Args: {'location': 'San Francisco'}
  Result: Sunny, 72Â°F

--- Iteration 2 ---
Assistant: The weather in San Francisco is sunny with a temperature of 72°F.


TEST 2: Query not requiring tool
User: Say hello!

--- Iteration 1 ---
Assistant: Hello! How can I assist you today?


TEST 3: Multiple tool calls
User: What's the weather in New York and London?

--- Iteration 1 ---
LLM wants to call 2 tool(s)
  Tool: get_weather
  Args: {'location': 'New York'}
  Result: Cloudy, 55Â°F
  Tool: get_weather
  Args: {'location': 'London'}
  Result: Rainy, 48Â°F

--- Iteration 2 ---
Assistant: The weather is currently as follows:

- **New York**: Cloudy, 55°F
- **London**: Rainy, 48°F



#Add the calculator you created for task 3 to it.  Create a tool that counts the number of occurences of a letter in a piece of text for answering questions like, "How may s are in Mississippi riverboats?".  Create a third tool of your own invention.  Record the output as you feed different inputs to the system and save it in your portfolio.  For good style, replace the tool execution dispatch code

In [23]:
"""
Tool Calling with LangChain
Shows how LangChain abstracts tool calling.
"""

from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
import math
import json

# ============================================
# PART 1: Define Your Tools
# ============================================

@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location"""
    # Simulated weather data
    weather_data = {
        "San Francisco": "Sunny, 72Â°F",
        "New York": "Cloudy, 55Â°F",
        "London": "Rainy, 48Â°F",
        "Tokyo": "Clear, 65Â°F"
    }
    return weather_data.get(location, f"Weather data not available for {location}")


@tool
def calculator(operation: str, operands: list) -> str:
    """
    Simple calculator with geometric functions.
    Returns JSON string.
    """
    try:
        if operation == "add":
            result = sum(operands)

        elif operation == "subtract":
            result = operands[0] - operands[1]

        elif operation == "multiply":
            result = operands[0] * operands[1]

        elif operation == "divide":
            result = operands[0] / operands[1]

        elif operation == "area_circle":
            r = operands[0]
            result = math.pi * r * r

        elif operation == "circumference_circle":
            r = operands[0]
            result = 2 * math.pi * r

        elif operation == "area_rectangle":
            w, h = operands
            result = w * h

        elif operation == "hypotenuse":
            a, b = operands
            result = math.sqrt(a * a + b * b)

        else:
            return json.dumps({"error": "Unknown operation"})

        return json.dumps({"result": result})

    except Exception as e:
        return json.dumps({"error": str(e)})


@tool
def count_letter(text: str, letter: str, case_sensitive: bool = False) -> str:
    """Count occurrences of a letter in text. Returns JSON string."""
    try:
        if not letter or len(letter) != 1:
            return json.dumps({"error": "letter must be a single character"})

        if case_sensitive:
            c = text.count(letter)
        else:
            c = text.lower().count(letter.lower())

        return json.dumps({"count": c, "letter": letter, "case_sensitive": case_sensitive})
    except Exception as e:
        return json.dumps({"error": str(e)})


@tool
def text_metrics(text: str) -> str:
    """Third tool (custom): basic text stats. Returns JSON string."""
    try:
        length_chars = len(text)
        word_count = len([w for w in text.split() if w.strip()])
        unique_letters = sorted({ch.lower() for ch in text if ch.isalpha()})
        return json.dumps({
            "length_chars": length_chars,
            "word_count": word_count,
            "unique_letter_count": len(unique_letters),
            "unique_letters": unique_letters
        })
    except Exception as e:
        return json.dumps({"error": str(e)})


# ============================================
# PART 2: Create LLM with Tools
# ============================================

# Create LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Bind tools to LLM
tools = [get_weather, calculator, count_letter, text_metrics]

llm_with_tools = llm.bind_tools(tools)

tool_map = {t.name: t for t in tools}



# ============================================
# PART 3: The Agent Loop
# ============================================

def run_agent(user_query: str):
    """
    Simple agent that can use tools.
    Shows the manual loop that LangGraph automates.
    """

    # Start conversation with user query
    messages = [
        SystemMessage(content="You are a helpful assistant. Use the provided tools when needed."),
        HumanMessage(content=user_query)
    ]

    print(f"User: {user_query}\n")

    # Agent loop - can iterate up to 5 times
    for iteration in range(5):
        print(f"--- Iteration {iteration + 1} ---")

        # Call the LLM
        response = llm_with_tools.invoke(messages)

        # Check if the LLM wants to call a tool
        if response.tool_calls:
            print(f"LLM wants to call {len(response.tool_calls)} tool(s)")

            # Add the assistant's response to messages
            messages.append(response)

            # Execute each tool call
            for tool_call in response.tool_calls:
                function_name = tool_call["name"]
                function_args = tool_call["args"]

                print(f"  Tool: {function_name}")
                print(f"  Args: {function_args}")

                # Execute the tool
                if function_name in tool_map:
                    result = tool_map[function_name].invoke(function_args)
                else:
                    result = f"Error: Unknown function {function_name}"

                print(f"  Result: {result}")
                # Add the tool result back to the conversation
                messages.append(ToolMessage(
                    content=result,
                    tool_call_id=tool_call["id"]
                ))

            print()
            # Loop continues - LLM will see the tool results

        else:
            # No tool calls - LLM provided a final answer
            print(f"Assistant: {response.content}\n")
            return response.content

    return "Max iterations reached"


# ============================================
# PART 4: Test It
# ============================================

if __name__ == "__main__":
    # Test query that requires tool use
    print("="*60)
    print("TEST 1: Query requiring tool")
    print("="*60)
    run_agent("What's the weather like in San Francisco?")

    print("\n" + "="*60)
    print("TEST 2: Query not requiring tool")
    print("="*60)
    run_agent("Say hello!")

    print("\n" + "="*60)
    print("TEST 3: Multiple tool calls")
    print("="*60)
    run_agent("What's the weather in New York and London?")

    print("\n" + "="*60)
    print("TEST 4: Multiple tool calls in the same turn (count twice)")
    print("="*60)
    run_agent("Are there more i's than s's in 'Mississippi riverboats'? Use tools.")

    print("\n" + "="*60)
    print("TEST 5: Sequential chaining (count twice → calculator next outer iteration)")
    print("="*60)
    run_agent("What is the sin of the difference between the number of i's and the number of s's in 'Mississippi riverboats'? Use tools.")

    print("\n" + "="*60)
    print("TEST 6: Use all tools in one query")
    print("="*60)
    run_agent("Using tools: (1) Get the weather in Tokyo. (2) Count the number of 's' in 'Mississippi riverboats'. (3) Compute text metrics for 'Mississippi riverboats'. (4) Compute the circumference of a circle with radius = count_s. Summarize everything.")

    print("\n" + "="*60)
    print("TEST 7: Try to hit the 5-iteration limit")
    print("="*60)
    run_agent("Call at most one tool per turn. Step 1: weather in London. Step 2: text_metrics for 'Mississippi riverboats'. Step 3: count 'i'. Step 4: count 's'. Step 5: compute sin(i-s). After each step, state what you learned and what you will do next.")

TEST 1: Query requiring tool
User: What's the weather like in San Francisco?

--- Iteration 1 ---
LLM wants to call 1 tool(s)
  Tool: get_weather
  Args: {'location': 'San Francisco'}
  Result: Sunny, 72Â°F

--- Iteration 2 ---
Assistant: The weather in San Francisco is sunny with a temperature of 72°F.


TEST 2: Query not requiring tool
User: Say hello!

--- Iteration 1 ---
Assistant: Hello! How can I assist you today?


TEST 3: Multiple tool calls
User: What's the weather in New York and London?

--- Iteration 1 ---
LLM wants to call 2 tool(s)
  Tool: get_weather
  Args: {'location': 'New York'}
  Result: Cloudy, 55Â°F
  Tool: get_weather
  Args: {'location': 'London'}
  Result: Rainy, 48Â°F

--- Iteration 2 ---
Assistant: The current weather is as follows:
- **New York**: Cloudy, 55°F
- **London**: Rainy, 48°F


TEST 4: Multiple tool calls in the same turn (count twice)
User: Are there more i's than s's in 'Mississippi riverboats'? Use tools.

--- Iteration 1 ---
LLM wants to call 2